Copyright **`(c)`** 2023 Alessandro Chiabodo `<s309234@studenti.polito.it>`  
[`https://github.com/AChiabodo/compIntelligence`](https://github.com/AChiabodo/compIntelligence)  
Free for personal use;


## LAB 2 - Nim

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

### Notes
The nim-sum can be used to "test" our algorithm given that it provides the "best" move to do in a given state. It should NOT be part of our algorithm, but it can be used to test it.

In [ ]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from itertools import product
import pickle

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    def non_zero_rows(self) -> list:
        return [True if x != 0 else False for x in self.rows]

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [ ]:
def generate_lists(numbers):
    n = len(numbers)

    all_combinations = list(product(range(0, max(numbers) + 1), repeat=n))

    valid_combinations = [list(combination) for combination in all_combinations if all(x <= y for x, y in zip(combination, numbers))]
    
    return valid_combinations[1:] #avoid the full zeros solution

In [ ]:
class Player():
    def play(self,state : Nim) -> Nimply :
        pass

    def __str__(self):
        return self.__class__.__name__
    
    def mutation(self,mutation_rate):
        pass

    def compute_fitness(self, state):
        pass

In [ ]:
def multiple_matches(player1 : Player,player2 : Player,matches = 100,size = 5):
    wins = [0,0]
    players = [player1,player2]
    for _ in range(matches):
        nim = Nim(size)
        i = random.choice([0,1])
        while nim :
            ply = players[i].play(nim)
            nim.nimming(ply)
            i = 1 - i
        wins[i] += 1
    return wins

In [ ]:
class PlayerNimStates(Player):

    def __init__(self,rules : dict) -> None:
        self.rules = rules
        self.fitness = None

    def play(self, state: Nim) -> Nimply:
        return self.rules[state.rows]
    
    def mutation(self,mutation_rate):
        for state in self.rules.keys():
            if random.random() < mutation_rate :
                self.rules[state] = random.choice([(r, o) for r, c in enumerate(state) for o in range(1, c + 1)])